In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
from seaborn import load_dataset
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [124]:
# sns.get_dataset_names('titanic')
data = load_dataset('titanic')
data.drop('alive',1,inplace=True)
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,True


In [125]:
from scipy.stats import mode
for col in data.columns:
#     print(data[col].value_counts().argmax())
    data[col].fillna(data[col].value_counts().argmax(),inplace=True)

In [126]:
data.isna().any()

survived       False
pclass         False
sex            False
age            False
sibsp          False
parch          False
fare           False
embarked       False
class          False
who            False
adult_male     False
deck           False
embark_town    False
alone          False
dtype: bool

In [127]:
cols = ['sex','embarked','class','who','adult_male','deck','embark_town','alone']
for col in cols:
    col_name = col+'_Labeller'
    print(col+'_Labeller')
    col_name = LabelEncoder()
    data[col] = col_name.fit_transform(data[col])
print('Labelling Done')

sex_Labeller
embarked_Labeller
class_Labeller
who_Labeller
adult_male_Labeller
deck_Labeller
embark_town_Labeller
alone_Labeller
Labelling Done


In [128]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone
0,0,3,1,22.0,1,0,7.2500,2,2,1,1,2,2,0
1,1,1,0,38.0,1,0,71.2833,0,0,2,0,2,0,0
2,1,3,0,26.0,0,0,7.9250,2,2,2,0,2,2,1
3,1,1,0,35.0,1,0,53.1000,2,0,2,0,2,2,0
4,0,3,1,35.0,0,0,8.0500,2,2,1,1,2,2,1


In [129]:
sc1  = StandardScaler()
sc2 = StandardScaler()
data['age'] = sc1.fit_transform(data.age.values.reshape(-1,1))
data['fare'] = sc1.fit_transform(data.fare.values.reshape(-1,1))

In [130]:
data.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alone'],
      dtype='object')

In [131]:
#Mean Encoding

cols = ['pclass', 'sex', 'sibsp', 'parch', 
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town','alone']
for col in cols:
    means = data.groupby(col)['survived'].mean()
    data[col+'_enc'] = data[col].map(means)
print('Mean Encoding Done')

Mean Encoding Done


In [132]:
#Count Encoding
cols = ['pclass', 'sex', 'sibsp', 'parch', 
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town','alone']
for col in cols:
#     freq = data[col].value_counts(dropna=False).to_dict() 
    data[col+'_freq'] = data[col].map(data[col].value_counts(dropna=False))
print('Count Encoding Done')

Count Encoding Done


In [133]:
y = data.survived
data.drop('survived',1,inplace=True)

In [134]:
categories = np.where(data.dtypes!=np.float)[0]
data=pd.get_dummies(data,columns=data.columns[categories].values)

In [135]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 96 columns):
age                     891 non-null float64
fare                    891 non-null float64
pclass_enc              891 non-null float64
sex_enc                 891 non-null float64
sibsp_enc               891 non-null float64
parch_enc               891 non-null float64
embarked_enc            891 non-null float64
class_enc               891 non-null float64
who_enc                 891 non-null float64
adult_male_enc          891 non-null float64
deck_enc                891 non-null float64
embark_town_enc         891 non-null float64
alone_enc               891 non-null float64
pclass_1                891 non-null uint8
pclass_2                891 non-null uint8
pclass_3                891 non-null uint8
sex_0                   891 non-null uint8
sex_1                   891 non-null uint8
sibsp_0                 891 non-null uint8
sibsp_1                 891 non-null uint8
sibsp_2  

In [136]:
X = data

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((668, 96), (223, 96), (668,), (223,))

In [139]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
import catboost as cat
# import lightgbm as lgt

In [140]:
model1 = ExtraTreesClassifier()
model2 = DecisionTreeClassifier()
model3 = RandomForestClassifier(n_estimators=1000)
model4 = SVC(C=10.0,probability=True)
model5 = LogisticRegression(C=1.0)
model6 = xgb.XGBClassifier(learning_rate=0.01, n_estimators=1000)
model7 = cat.CatBoostClassifier(iterations=1000, learning_rate=0.01)

In [141]:
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)
model4.fit(X_train, y_train)

SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [142]:
pred1 = model1.predict(X_test)
pred2 = model2.predict(X_test)
pred3 = model3.predict(X_test)
pred4 = model4.predict(X_test)
print(model1.score(X_test,y_test),model2.score(X_test,y_test),model3.score(X_test,y_test),model4.score(X_test,y_test))

0.7802690582959642 0.7802690582959642 0.7757847533632287 0.820627802690583


In [143]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier(estimators=[('ET',model1),('DT',model2),('RF',model3),('SVC',model4)],voting='hard')
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7982062780269058

Averaging

In [144]:
preds1 = model1.predict_proba(X_test)
preds2 = model2.predict_proba(X_test)
preds3 = model3.predict_proba(X_test)
preds4 = model4.predict_proba(X_test)

finalpred = (preds2+preds3+preds4)/4
finalpred_C = np.where(finalpred[:,1]>0.5,1,0)
finalpred_C
confusion_matrix(finalpred_C,y_test),accuracy_score(y_test,finalpred_C)

(array([[123,  29],
        [ 11,  60]], dtype=int64), 0.820627802690583)

Weighted_Averaging

In [145]:
preds1 = model1.predict(X_test)
preds2 = model2.predict(X_test)
preds3 = model3.predict(X_test)
preds4 = model4.predict(X_test)

In [146]:
print(roc_auc_score(y_test, preds1))
print(roc_auc_score(y_test, preds2))
print(roc_auc_score(y_test, preds3))  
print(roc_auc_score(y_test, (preds1 + preds3)*0.5))  
print(roc_auc_score(y_test, (preds1 + preds2 + preds3*0.5)*0.33))
print(roc_auc_score(y_test, (preds1*0.5 + preds2*0.5 + preds3 + preds4*0.5)*0.33))

0.7737715914807982
0.7794314942143216
0.7738135166862318
0.785594499413047
0.826387724299849
0.851542847559953


In [147]:
(preds1*0.5 + preds2*0.5 + preds3 + preds4*0.5)*0.33

array([0.66 , 0.165, 0.   , 0.825, 0.33 , 0.825, 0.825, 0.   , 0.66 ,
       0.825, 0.   , 0.   , 0.   , 0.   , 0.   , 0.825, 0.165, 0.825,
       0.   , 0.   , 0.   , 0.495, 0.   , 0.   , 0.   , 0.   , 0.165,
       0.165, 0.   , 0.495, 0.   , 0.825, 0.66 , 0.825, 0.   , 0.   ,
       0.   , 0.825, 0.825, 0.   , 0.   , 0.165, 0.   , 0.   , 0.165,
       0.165, 0.33 , 0.   , 0.165, 0.495, 0.66 , 0.825, 0.   , 0.825,
       0.   , 0.825, 0.165, 0.825, 0.66 , 0.33 , 0.33 , 0.825, 0.825,
       0.   , 0.   , 0.825, 0.   , 0.   , 0.66 , 0.825, 0.825, 0.825,
       0.66 , 0.825, 0.   , 0.   , 0.825, 0.825, 0.825, 0.33 , 0.165,
       0.825, 0.825, 0.   , 0.66 , 0.165, 0.825, 0.825, 0.   , 0.   ,
       0.825, 0.   , 0.   , 0.   , 0.   , 0.   , 0.165, 0.   , 0.825,
       0.   , 0.   , 0.   , 0.825, 0.   , 0.   , 0.66 , 0.825, 0.495,
       0.   , 0.495, 0.825, 0.   , 0.825, 0.66 , 0.   , 0.   , 0.66 ,
       0.66 , 0.825, 0.   , 0.   , 0.825, 0.825, 0.825, 0.   , 0.   ,
       0.825, 0.66 ,

In [148]:
pred1 = model1.predict_proba(X_test)
pred2 = model2.predict_proba(X_test)
pred3 = model3.predict_proba(X_test)
pred4 = model4.predict_proba(X_test)

final_pred = ((pred1*0.5 + pred2*0.5 + pred3 + pred4*0.5)*0.33)
# final_pred = (preds2*0.5 + preds3 + preds4*0.5)*0.33
final_pred

array([[0.32371842, 0.50128158],
       [0.63197788, 0.19302212],
       [0.79145103, 0.03354897],
       [0.02491613, 0.80008387],
       [0.41798205, 0.40701795],
       [0.02542646, 0.79957354],
       [0.13396175, 0.69103825],
       [0.78164453, 0.04335547],
       [0.34784674, 0.47715326],
       [0.04419314, 0.78080686],
       [0.7286349 , 0.0963651 ],
       [0.76945511, 0.05554489],
       [0.76779537, 0.05720463],
       [0.78258156, 0.04241844],
       [0.79774106, 0.02725894],
       [0.01953098, 0.80546902],
       [0.52807199, 0.29692801],
       [0.03814495, 0.78685505],
       [0.79804993, 0.02695007],
       [0.75027314, 0.07472686],
       [0.79948545, 0.02551455],
       [0.39069956, 0.43430044],
       [0.71811735, 0.10688265],
       [0.79951185, 0.02548815],
       [0.79815703, 0.02684297],
       [0.73213357, 0.09286643],
       [0.61422704, 0.21077296],
       [0.63226695, 0.19273305],
       [0.70533861, 0.11966139],
       [0.46136569, 0.36363431],
       [0.

In [149]:
final_pred_C = np.where(final_pred[:,1]>0.5,1,0)
final_pred_C
confusion_matrix(final_pred_C,y_test),accuracy_score(y_test,final_pred_C)

(array([[117,  25],
        [ 17,  64]], dtype=int64), 0.8116591928251121)

Stacking

In [150]:
def Stacking(model, train,y, test, n_fold):
    folds = StratifiedKFold(n_splits=n_fold, random_state=1)
    test_pred = np.empty((0,1),float)
    train_pred = np.empty((0,1),float)
    for train_indices, val_indices in folds.split(train,y.values):
        print(train_indices.shape, val_indices.shape)
        x_train, x_val = train.iloc[train_indices], train.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]   
        model.fit(X=x_train, y=y_train)
        train_pred = np.append(train_pred, model.predict(x_val))
        print('train_pred_shape',train_pred.shape)
    test_pred = np.append(test_pred, model.predict(test))
    print('test_pred_shape',test_pred.shape)
    return test_pred.reshape(-1,1), train_pred

In [151]:
model1 = DecisionTreeClassifier(random_state=42)

test_pred1, train_pred1 = Stacking(model=model1, n_fold=10, train=X_train, test = X_test, y=y_train)

train_pred1 = pd.DataFrame(train_pred1)
test_pred1 = pd.DataFrame(test_pred1)
# train_pred1

(600,) (68,)
train_pred_shape (68,)
(600,) (68,)
train_pred_shape (136,)
(600,) (68,)
train_pred_shape (204,)
(601,) (67,)
train_pred_shape (271,)
(601,) (67,)
train_pred_shape (338,)
(602,) (66,)
train_pred_shape (404,)
(602,) (66,)
train_pred_shape (470,)
(602,) (66,)
train_pred_shape (536,)
(602,) (66,)
train_pred_shape (602,)
(602,) (66,)
train_pred_shape (668,)
test_pred_shape (223,)


In [152]:
model2 = KNeighborsClassifier()

test_pred2, train_pred2 = Stacking(model=model1, n_fold=10, train=X_train, test = X_test, y=y_train)

train_pred2 = pd.DataFrame(train_pred2)
test_pred2 = pd.DataFrame(test_pred2)

(600,) (68,)
train_pred_shape (68,)
(600,) (68,)
train_pred_shape (136,)
(600,) (68,)
train_pred_shape (204,)
(601,) (67,)
train_pred_shape (271,)
(601,) (67,)
train_pred_shape (338,)
(602,) (66,)
train_pred_shape (404,)
(602,) (66,)
train_pred_shape (470,)
(602,) (66,)
train_pred_shape (536,)
(602,) (66,)
train_pred_shape (602,)
(602,) (66,)
train_pred_shape (668,)
test_pred_shape (223,)


In [153]:
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

model = LogisticRegression(random_state=42)
model.fit(df, y_train)
model.score(df_test, y_test)

0.7623318385650224

KFold Working

In [154]:
n_fold = 5
folds = KFold(n_splits=n_fold,shuffle=True)
print(folds)

KFold(n_splits=5, random_state=None, shuffle=True)


In [155]:
for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train)):
    print(train_index)

[  0   1   2   3   6   9  10  12  13  14  15  16  17  18  19  20  21  22
  23  24  25  26  27  28  29  30  31  32  33  34  37  38  39  40  42  44
  47  49  51  52  53  54  55  56  57  58  59  60  61  64  65  66  67  69
  70  71  72  73  74  76  77  78  79  80  81  82  83  85  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 104 105 106 108 109 110
 111 112 113 114 115 116 118 119 120 121 122 123 124 126 127 128 130 131
 132 133 134 135 136 137 138 139 141 143 144 145 147 149 150 151 152 153
 154 155 156 157 158 159 160 161 162 163 164 165 166 167 169 170 171 174
 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 193
 194 195 197 198 200 201 202 203 204 205 206 208 213 214 215 216 217 218
 219 220 221 222 224 225 226 228 229 231 232 233 234 235 236 237 239 241
 242 243 244 245 246 247 249 250 251 254 255 256 257 258 259 261 262 263
 264 266 267 268 269 271 273 274 276 277 279 280 282 283 284 285 286 287
 288 289 291 292 293 294 295 296 298 300 302 303 30

Blending

In [156]:
x_train,x_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2, random_state=42)
x_train.shape,x_val.shape,y_train.shape,y_val.shape

((534, 96), (134, 96), (534,), (134,))

In [157]:
model1 = DecisionTreeClassifier(random_state=42)

model1.fit(x_train,y_train)

val_pred1 = model1.predict(x_val)
test_pred1 = model1.predict(X_test)

val_pred1 = pd.DataFrame(val_pred1,columns=['first'])
test_pred1 = pd.DataFrame(test_pred1,columns=['first'])

model2 = KNeighborsClassifier()

model2.fit(x_train,y_train)

val_pred2 = model1.predict(x_val)
test_pred2 = model1.predict(X_test)

val_pred2 = pd.DataFrame(val_pred2,columns=['second'])
test_pred2 = pd.DataFrame(test_pred2,columns=['second'])

In [158]:
X_test.shape,test_pred1.shape, test_pred2.shape

((223, 96), (223, 1), (223, 1))

In [159]:
x_val.shape, val_pred1.shape, val_pred2.shape

((134, 96), (134, 1), (134, 1))

In [160]:
#Combining the meta features and validation set - Logistic regression

df_val = pd.concat([x_val.reset_index(), val_pred1, val_pred2],axis=1,)
df_test = pd.concat([X_test.reset_index(), test_pred1, test_pred2],axis=1)

model = LogisticRegression()
model.fit(df_val,y_val)
model.score(df_test, y_test)

from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=42,n_estimators=500,learning_rate=0.01)
model.fit(df_val,y_val)
model.score(df_test, y_test)


from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=42,n_estimators=5000,learning_rate=0.01,loss='exponential')
model.fit(df_val,y_val)
model.score(df_test, y_test)


import xgboost as xgb
model = xgb.XGBClassifier(random_state=42,learning_rate=0.01)
model.fit(df_val,y_val)
model.score(df_test, y_test)

0.7892376681614349

Bagging

In [162]:
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(DecisionTreeClassifier(random_state=42))
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.7847533632286996

In [163]:
model = RandomForestClassifier(random_state=42,n_estimators=1000)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.7623318385650224

Boosting

In [164]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=42,n_estimators=500,learning_rate=0.01)
model.fit(df_val,y_val)
model.score(df_test, y_test)

0.8251121076233184

In [167]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=42,n_estimators=5000,learning_rate=0.01,loss='exponential')
model.fit(df_val,y_val)
model.score(df_test, y_test)

0.7668161434977578

In [168]:
import xgboost as xgb
model = xgb.XGBClassifier(random_state=42,learning_rate=0.01,n_estimators=5000)
model.fit(df_val,y_val)
model.score(df_test, y_test)

0.7937219730941704

In [169]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train,label=y_train)

#define parameters
params = {'learning_rate':0.001}
model = lgb.train(params, train_data, 100)
y_pred = model.predict(X_test)
y_pred = np.where(y_pred>0.4,1,0)
accuracy_score(y_test,y_pred)

0.8161434977578476

In [170]:
import catboost as cat
model = cat.CatBoostClassifier(use_best_model=True, eval_metric='Accuracy',depth= 4, iterations= 51, learning_rate= 0.07483533251216079)
categorical_feature_indices = np.where(df.dtypes!=np.float)[0]
model.fit(X_train, y_train, cat_features=categorical_feature_indices, eval_set=(X_test,y_test))
model.score(X_test, y_test)

0:	learn: 0.8053892	test: 0.7802691	best: 0.7802691 (0)	total: 22.3ms	remaining: 1.11s
1:	learn: 0.8143713	test: 0.8161435	best: 0.8161435 (1)	total: 50.2ms	remaining: 1.23s
2:	learn: 0.8143713	test: 0.8161435	best: 0.8161435 (1)	total: 66.6ms	remaining: 1.06s
3:	learn: 0.8188623	test: 0.8161435	best: 0.8161435 (1)	total: 86.3ms	remaining: 1.01s
4:	learn: 0.8233533	test: 0.8116592	best: 0.8161435 (1)	total: 128ms	remaining: 1.18s
5:	learn: 0.8233533	test: 0.8116592	best: 0.8161435 (1)	total: 156ms	remaining: 1.17s
6:	learn: 0.8233533	test: 0.8206278	best: 0.8206278 (6)	total: 188ms	remaining: 1.18s
7:	learn: 0.8263473	test: 0.8116592	best: 0.8206278 (6)	total: 208ms	remaining: 1.12s
8:	learn: 0.8263473	test: 0.8206278	best: 0.8206278 (6)	total: 234ms	remaining: 1.09s
9:	learn: 0.8263473	test: 0.8116592	best: 0.8206278 (6)	total: 253ms	remaining: 1.04s
10:	learn: 0.8278443	test: 0.8116592	best: 0.8206278 (6)	total: 269ms	remaining: 979ms
11:	learn: 0.8278443	test: 0.8116592	best: 0.8206

0.8251121076233184

GridsearchCV with Catboost

In [171]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

model = cat.CatBoostClassifier()
parameters = {
    'depth':sp_randInt(4,10),
    'learning_rate':sp_randFloat(),
    'iterations':sp_randInt(10,100)
}

randm = RandomizedSearchCV(estimator=model, param_distributions=parameters, cv=2, n_iter=10, n_jobs=-1, random_state=42)
randm.fit(X_train,y_train)
pred = randm.predict(X_test)
print(accuracy_score(y_test, pred))

print(' best estimator - ', randm.best_estimator_)
print(' best score - ', randm.best_score_)
print(' best params - ', randm.best_params_)

0:	learn: 0.6926300	total: 59.6ms	remaining: 2.74s
1:	learn: 0.6921075	total: 77.3ms	remaining: 1.74s
2:	learn: 0.6915390	total: 151ms	remaining: 2.21s
3:	learn: 0.6909394	total: 194ms	remaining: 2.08s
4:	learn: 0.6903608	total: 262ms	remaining: 2.2s
5:	learn: 0.6898199	total: 298ms	remaining: 2.04s
6:	learn: 0.6892510	total: 361ms	remaining: 2.06s
7:	learn: 0.6886986	total: 401ms	remaining: 1.96s
8:	learn: 0.6881436	total: 473ms	remaining: 2s
9:	learn: 0.6875948	total: 498ms	remaining: 1.84s
10:	learn: 0.6870773	total: 565ms	remaining: 1.85s
11:	learn: 0.6865367	total: 648ms	remaining: 1.89s
12:	learn: 0.6860251	total: 670ms	remaining: 1.75s
13:	learn: 0.6856241	total: 723ms	remaining: 1.7s
14:	learn: 0.6851050	total: 770ms	remaining: 1.64s
15:	learn: 0.6845189	total: 842ms	remaining: 1.63s
16:	learn: 0.6839438	total: 922ms	remaining: 1.63s
17:	learn: 0.6834451	total: 959ms	remaining: 1.54s
18:	learn: 0.6829125	total: 995ms	remaining: 1.47s
19:	learn: 0.6823459	total: 1.03s	remaining:

Vecstack - Stacking library

In [122]:
from vecstack import stacking

models = [
    KNeighborsClassifier(n_neighbors=5,
                        n_jobs=-1),
        
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=600, max_depth=3),
        
    xgb.XGBClassifier(random_state=0, subsample=1.0, n_estimators= 600, min_child_weight= 1, max_depth = 10, learning_rate= 0.001, gamma= 0.5, eta= 0.1, colsample_bytree= 1.0)
]

S_train, S_test = stacking(models,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.78571429]
    fold  1:  [0.80239521]
    fold  2:  [0.76047904]
    fold  3:  [0.83734940]
    ----
    MEAN:     [0.79648448] + [0.02791624]
    FULL:     [0.79640719]

model  1:     [RandomForestClassifier]
    fold  0:  [0.80952381]
    fold  1:  [0.79640719]
    fold  2:  [0.79640719]
    fold  3:  [0.85542169]
    ----
    MEAN:     [0.81443997] + [0.02425919]
    FULL:     [0.81437126]

model  2:     [XGBClassifier]
    fold  0:  [0.83928571]
    fold  1:  [0.82035928]
    fold  2:  [0.79640719]
    fold  3:  [0.83734940]
    ----
    MEAN:     [0.82335039] + [0.01721039]
    FULL:     [0.82335329]



In [123]:
model = xgb.XGBClassifier(random_state=0, n_jobs=-1, subsample=1.0, n_estimators= 600, min_child_weight= 1, max_depth = 10, learning_rate= 0.001, gamma= 0.5, eta= 0.1, colsample_bytree= 1.0)
    
model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score: [0.82062780]


XGboost with gridsearchcv

In [116]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5,6,7,10],
        'eta':[.3, .2, .1, .05, .01, .005],
        'learning_rate':[1.0,0.1,0.5,0.005,0.001],
        'n_estimators':[100,500,600,700,900,1000]
        }

xgbClass = xgb.XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic')

folds = 10
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgbClass, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=skf.split(data,y), verbose=3, random_state=1001 )

random_search.fit(X, y)

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  44 out of  50 | elapsed:   14.6s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   15.4s finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x00000138E5C091B0>,
                   error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.02, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=600,
                                           n_jobs=1, nthread=None,
                                           o...
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'eta': [0.3, 0.2, 0.1, 0.05, 0.01,
                                                0.005],
                                    

In [117]:
print('\n Best hyperparameters:')
print(random_search.best_params_)


 Best hyperparameters:
{'subsample': 1.0, 'n_estimators': 600, 'min_child_weight': 1, 'max_depth': 10, 'learning_rate': 0.001, 'gamma': 0.5, 'eta': 0.1, 'colsample_bytree': 1.0}


In [121]:
model = xgb.XGBClassifier(random_state=42,subsample=1.0, n_estimators= 600, min_child_weight= 1, max_depth = 10, learning_rate= 0.001, gamma= 0.5, eta= 0.1, colsample_bytree= 1.0)
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.8295964125560538